## Initialisation

In [1]:
import zarr
import zarrita
import numpy as np

In [2]:
store = zarrita.LocalStore('testoutput')

testdata = np.arange(0, 16 * 16, dtype='int32').reshape((16, 16))

## Creating an array

In [13]:
a = zarrita.Array.create(
    store='array',
    shape=(16, 16),
    dtype='int32',
    chunk_shape=(4, 4),
    codecs=[
        zarrita.codecs.bytes_codec(),
        zarrita.codecs.blosc_codec(typesize=4),
    ],
    attributes={'question': 'life', 'answer': 42}
)
a[:, :] = testdata

In [15]:
!tree -a array | head

array
├── .DS_Store
├── c
│   ├── .DS_Store
│   ├── 0
│   │   ├── 0
│   │   ├── 1
│   │   ├── 2
│   │   └── 3
│   ├── 1


## Looking at the array's metadata

In [16]:
import json
with open('array/zarr.json') as fp:
    print(json.load(fp))

{'shape': [16, 16], 'data_type': 'int32', 'chunk_grid': {'configuration': {'chunk_shape': [4, 4]}, 'name': 'regular'}, 'chunk_key_encoding': {'configuration': {'separator': '/'}, 'name': 'default'}, 'fill_value': 0, 'codecs': [{'configuration': {'endian': 'little'}, 'name': 'bytes'}, {'configuration': {'typesize': 4, 'cname': 'zstd', 'clevel': 5, 'shuffle': 'noshuffle', 'blocksize': 0}, 'name': 'blosc'}], 'attributes': {'question': 'life', 'answer': 42}, 'zarr_format': 3, 'node_type': 'array'}


## Opening an array

In [17]:
a = zarrita.Array.open(store='array')
assert np.array_equal(a[:, :], testdata)

## Creating a [Sharded](https://zarr.dev/zeps/accepted/ZEP0002.html) Array

In [25]:
a = zarrita.Array.create(
    store='sharding',
    shape=(16, 16),
    dtype='int32',
    chunk_shape=(8, 8),
    chunk_key_encoding=('v2', '.'),
    codecs=[
        zarrita.codecs.sharding_codec(
            chunk_shape=(4, 4),
            codecs=[
                zarrita.codecs.bytes_codec(),
                zarrita.codecs.blosc_codec(typesize=4),
            ]
        ),
    ],
)
a[:, :] = testdata
assert np.array_equal(a[:, :], testdata)

In [26]:
!tree -a sharding | head

sharding
├── 0.0
├── 0.1
├── 1.0
├── 1.1
└── zarr.json

0 directories, 5 files


## Looking at the array's metadata

In [30]:
with open('sharding/zarr.json') as fp:
    print(json.load(fp))

{'shape': [16, 16], 'data_type': 'int32', 'chunk_grid': {'configuration': {'chunk_shape': [8, 8]}, 'name': 'regular'}, 'chunk_key_encoding': {'configuration': {'separator': '.'}, 'name': 'v2'}, 'fill_value': 0, 'codecs': [{'configuration': {'chunk_shape': [4, 4], 'codecs': [{'configuration': {'endian': 'little'}, 'name': 'bytes'}, {'configuration': {'typesize': 4, 'cname': 'zstd', 'clevel': 5, 'shuffle': 'noshuffle', 'blocksize': 0}, 'name': 'blosc'}], 'index_codecs': [{'configuration': {'endian': 'little'}, 'name': 'bytes'}, {'name': 'crc32c'}], 'index_location': 'end'}, 'name': 'sharding_indexed'}], 'attributes': {}, 'zarr_format': 3, 'node_type': 'array'}


## Creating a group

In [32]:
g = zarrita.Group.create(store='group')
g2 = g.create_group('group2')
a = g2.create_array(
    'array',
    shape=(16, 16),
    dtype='int32',
    chunk_shape=(16, 16),
)
a[:, :] = testdata

In [33]:
!tree -a group | head

group
├── group2
│   ├── array
│   │   ├── c
│   │   │   └── 0
│   │   │       └── 0
│   │   └── zarr.json
│   └── zarr.json
└── zarr.json



## Navigating inside the group

In [34]:
with open('group/zarr.json') as fp:
    print(json.load(fp))

{'attributes': {}, 'zarr_format': 3, 'node_type': 'group'}


In [35]:
with open('group/group2/zarr.json') as fp:
    print(json.load(fp))

{'attributes': {}, 'zarr_format': 3, 'node_type': 'group'}


In [36]:
with open('group/group2/array/zarr.json') as fp:
    print(json.load(fp))

{'shape': [16, 16], 'data_type': 'int32', 'chunk_grid': {'configuration': {'chunk_shape': [16, 16]}, 'name': 'regular'}, 'chunk_key_encoding': {'configuration': {'separator': '/'}, 'name': 'default'}, 'fill_value': 0, 'codecs': [{'configuration': {'endian': 'little'}, 'name': 'bytes'}], 'attributes': {}, 'zarr_format': 3, 'node_type': 'array'}


## Opening a group

In [29]:
g = zarrita.Group.open(store='group')
g2 = g['group2']
a = g['group2']['array']
assert np.array_equal(a[:, :], testdata)